In [1]:
from pylyra import *

# -- Metrics --------

print("Definindo metrica")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)
dim = 4

f1 = sp.Function('f1')(r)
f2 = sp.Function('f2')(r)

g = sp.diag(sp.exp(f1), -sp.exp(f2), -r**2, -r**2*sp.sin(th)**2)  # g_{mu nu}
g_inv = sp.simplify(g.inv())

detg = sp.simplify(g.det())
sqrt_minus_detg = sp.sqrt(-detg)

# -- Espaço Tensorial ----------
space = TensorSpace(dim=4, coords=x, metric=g, metric_inv=g_inv)

# -- Escala de Lyra ----------
print("Definindo escala")
phi = sp.Function('phi')(r)

# -- Tensor de Levi Civita ----------
print("Definindo LeviCivita")
eps = space.from_function(
    lambda mu, nu, rho, sig: sqrt_minus_detg * sp.LeviCivita(mu, nu, rho, sig),
    signature=(d, d, d, d),
)

# -- Christoffel ----------
print("Definindo Christoffel")

def create_christoffel_array():
    dim = g.shape[0]
    chris = [[[
        (sp.Rational(1, 2) * sum(
            g_inv[a, d] * (sp.diff(g[d, b], x[c]) +
                           sp.diff(g[d, c], x[b]) -
                           sp.diff(g[b, c], x[d]))
            for d in range(dim)
        ))
        for c in range(dim)]
        for b in range(dim)]
        for a in range(dim)]
    return sp.Array(chris)

chris = create_christoffel_array()

# -- Non-Metricity ----------
print("Definindo Non-Metricity")
M = space.from_function(
    lambda a, b, c: 0,
    signature=(u, d, d),
)

# -- Torsion ----------
print("Definindo Torsion")

def torsion_axial(mu):
    if mu == 0:
        return sp.Function('s_t')(r)
    if mu == 1:
        return sp.Function('s_r')(r)
    return 0

s = space.from_function(torsion_axial, signature=(u,))


def torsion_trace(mu):
    if mu == 0:
        return sp.Function('v_t')(r)
    if mu == 1:
        return sp.Function('v_r')(r)
    return 0

v = space.from_function(torsion_trace, signature=(d,))


def torsion_component(m, n, r_):
    return (
        sum(eps(d, d, d, d).comp[m, n, r_, sig] * s(u).comp[sig] for sig in range(dim))
        + sp.Rational(1, 3) * (
            g[m, n] * v(d).comp[r_]
            - g[m, r_] * v(d).comp[n]
        )
    )

tau = space.from_function(torsion_component, signature=(d, d, d))

# -- Connection ----------
print("Definindo Connection")

def connection_element(b, n, l):
    return (
        1 / phi * chris[b, n, l]
        - sp.Rational(1, 2) * M(u, d, d)[b, n, l]
        + 1 / (phi) * (
            sp.KroneckerDelta(b, n) * 1 / phi * sp.diff(phi, x[l])
            - sum(1 / phi * g[n, l] * g_inv[b, s] * sp.diff(phi, x[s]) for s in range(dim))
        )
        + sp.Rational(1, 2) * sum(
            g_inv[m, b] * (
                tau(d, d, d)[l, m, n] - tau(d, d, d)[n, l, m] - tau(d, d, d)[m, l, n]
            )
            for m in range(dim)
        )
    )

Gamma = table(connection_element, dim=dim)
space.set_connection(Gamma)

# -- Curvature ----------
print("Definindo Curvature")

def curvature_element(l, a, m, n):
    return (
        1 / (phi**2) * sp.diff(phi * Gamma[l, a, n], x[m])
        - 1 / (phi**2) * sp.diff(phi * Gamma[l, a, m], x[n])
        + sum(Gamma[r, a, n] * Gamma[l, r, m] for r in range(dim))
        - sum(Gamma[r, a, m] * Gamma[l, r, n] for r in range(dim))
    )

Curv = space.from_function(curvature_element, signature=(u, d, d, d))

# -- Ricci ----------
print("Definindo Ricci")

def ricci_element(a, m):
    return sp.expand(sp.simplify(
        sum(Curv(u, d, d, d).comp[l, a, m, l] for l in range(dim))
    ))

Ricc = space.from_function(ricci_element, signature=(d, d))

# -- Curvature Scalar ----------
print("Definindo Curvature Scalar")
R = sp.expand(sum(Ricc(u, d).comp[l, l] for l in range(dim))).subs(th, sp.pi/2)


Definindo metrica
Definindo escala
Definindo LeviCivita
Definindo Christoffel
Definindo Non-Metricity
Definindo Torsion
Definindo Connection
Definindo Curvature
Definindo Ricci
Definindo Curvature Scalar


In [2]:
from pylyra import *
# supondo que space já tenha coords e connection (Gamma) definidos
T = space.from_function(lambda a, b: sp.Function("T_%s%s" % (a,b))(*space.coords),
                        signature=(u, d))

# adiciona o índice covariante no fim: (u, d, d)
dT = T.nabla(deriv_position="append")

# componente: dT[a,b,k] = ∇_k T^a_b
dT.comp[0, 1, 2]


r**2*T_21(t, r, theta, varphi)*v_t(r)*exp(-f1(r))/3 + r**2*T_31(t, r, theta, varphi)*s_r(r)*sqrt(exp(f1(r) + f2(r)))*exp(-f1(r))*Abs(sin(theta))/2 - T_02(t, r, theta, varphi)*v_r(r)/3 - T_02(t, r, theta, varphi)*Derivative(phi(r), r)/phi(r)**2 + T_03(t, r, theta, varphi)*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))/(2*sin(theta)**2) + Derivative(T_01(t, r, theta, varphi), theta) - T_02(t, r, theta, varphi)/(r*phi(r))

Na verdade, queria melhorar as abstrações. Da forma como está agora:

- A métrica não é um tensor
- Escalares não são tensores
- Ter alguma forma de fazer contrações


Pensei em ter um elemento que abstraia o espaço-tempo. Manifold? Não sei

Dai, deveriamos ter métodos set_scale. Se não passar, usa-se phi = sp.Function('phi')(r)

um método set metric



In [ ]:
M.contract(
    
)